# Enron Tensor Flow

From [Amunategui](http://amunategui.github.io/office-automation/index.html)


In [1]:
import tensorflow as tf
import collections
import os
import random
import numpy as np
from tqdm import tqdm
import sys, email
import pandas as pd 
import math
from pathlib import Path

2022-03-17 16:23:29.618195: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mhc/Programs/freeling/lib:/home/mhc/Programs/freeling/share/freeling/APIs/python3
2022-03-17 16:23:29.618242: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
%%time
ENRON_PATH = Path('..','emails.csv')

emails_df = pd.read_csv(ENRON_PATH)

In [3]:
emails_df.shape

(517401, 2)

In [8]:
def get_text_from_email(msg):
    """To get the content from email objects"""
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append(part.get_payload())
    
    return ''.join(parts)


def extract_text(message):
    """
    Parses the last message in case there are multiple replies.
    """
    
    msg = email.message_from_string(message)
    msg = msg.get_payload()
    
    return msg


def extract_item(message, item):
    """
    Extract message item
    """
    
    msg = email.message_from_string(message)
    
    return msg[item]

In [7]:
%%time
emails_df['content'] = emails_df['message'].apply(extract_text)

CPU times: user 57.7 s, sys: 279 ms, total: 57.9 s
Wall time: 57.9 s


In [10]:
%%time
emails_df['subject'] = emails_df['message'].apply(extract_item, args=('Subject',))
emails_df['date'] = emails_df['message'].apply(extract_item, args=('Date',))

CPU times: user 1min 55s, sys: 83.6 ms, total: 1min 55s
Wall time: 1min 55s


In [12]:
emails_df.drop('message', axis=1, inplace=True)

In [15]:
emails_df['content'] = emails_df['content'].str.lower()
emails_df['content'] = emails_df['content'].str.replace(r'[^a-z]', ' ')  
emails_df['content'] = emails_df['content'].str.replace(r'\s+', ' ')  

/tmp/ipykernel_9810/651153915.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  emails_df['content'] = emails_df['content'].str.replace(r'[^a-z]', ' ')
/tmp/ipykernel_9810/651153915.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  emails_df['content'] = emails_df['content'].str.replace(r'\s+', ' ')


In [16]:
emails_df['content']

0                                     here is our forecast 
1         traveling to have a business meeting takes the...
2                                test successful way to go 
3         randy can you send me a schedule of the salary...
4                               let s shoot for tuesday at 
                                ...                        
517396    this is a trade with oil spec hedge ng john la...
517397    some of my position is with the alberta term b...
517398     original message from doucet dawn sent wednes...
517399    analyst rank stephane brodeur chad clark ian c...
517400    i think the ymca has a class that is for peopl...
Name: content, Length: 517401, dtype: object